In [1]:
import xarray as xr
import numpy as np
import xmitgcm
from matplotlib import pyplot as plt
import os
from glob import glob
%matplotlib inline
from matplotlib.colors import LogNorm
import dask
from dask.diagnostics import ProgressBar
from xgcm import Grid

In [2]:
plt.rcParams['figure.figsize'] = (15,10)

plt.rcParams.update({'font.size': 18
    , 'legend.markerscale': 1., 'axes.titlesize': 18, 'axes.labelsize' : 18,
      'legend.fontsize' : 14,'legend.handlelength': 3})

plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

In [ ]:
ddir = '/swot/SUM01/LLC/llc_4320_agulhas/'
all_files = sorted(glob(os.path.join(ddir, 'llc_4320_agulhas.0*.nc')))
print(len(all_files))
all_files[0], all_files[-1]

In [4]:
ds = xr.open_mfdataset(all_files[:24*70], decode_cf=False, autoclose=True, chunks={'k': 1, 'k_l': 1, 'time': 1})
#ds = xr.open_dataset(all_files[], decode_cf=False, chunks={'k': 1, 'k_l': 1})
ds = ds.set_coords(['iter', 'face'])

In [5]:
grid_ds = xr.open_dataset(ddir + 'llc_4320_agulhas_grid.nc')
ds = xr.merge([grid_ds, ds])
ds = xmitgcm.mds_store._swap_dimensions(ds, geometry='sphericalpolar')
#ds_sec = ds.isel(YC = slice(1300,1500), YG = slice(1300,1500), XC = slice(1000,1200), XG = slice(1000,1200), Z=0, Zl=0,Zp1=0,Zu=0 )
ds_sec = ds.isel(YC = slice(970,1650), XC = slice(985,1465), YG = slice(970,1650), XG = slice(985,1465), Z=0, Zl=0,Zp1=0,Zu=0  )
ds = ds_sec

grid = Grid(ds, periodic=False)
grid

<xgcm.Grid>
Y Axis (not periodic):
  * center   YC (680) --> left
  * left     YG (680) --> center
X Axis (not periodic):
  * center   XC (480) --> left
  * left     XG (480) --> center
T Axis (not periodic):
  * center   time (1680)

In [7]:
ds

<xarray.Dataset>
Dimensions:   (XC: 480, XG: 480, YC: 680, YG: 680, time: 1680)
Coordinates:
  * YC        (YC) float32 -45.047726 -45.033997 -45.020264 -45.006527 ...
  * YG        (YG) float32 -45.054592 -45.040863 -45.02713 -45.013397 ...
  * XC        (XC) float32 5.03125 5.0520835 5.0729165 5.09375 5.1145835 ...
  * XG        (XG) float32 5.0208335 5.0416665 5.0625 5.0833335 5.1041665 ...
    Zp1       float32 0.0
    Z         float32 -0.5
    Zl        float32 0.0
    Zu        float32 -1.0
  * time      (time) float64 2.592e+05 2.628e+05 2.664e+05 2.7e+05 2.736e+05 ...
Data variables:
    face      (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
    rA        (YC, XC) float32 ...
    rAw       (YC, XG) float32 ...
    rAs       (YG, XC) float32 ...
    rAz       (YG, XG) float32 ...
    dxG       (YG, XC) float32 ...
    dyG       (YC, XG) float32 ...
    dxC       (YC, XG) float32 ...
    Depth     (YC, XC) float32 ...
    dyC       (YG, XC) float32 ...
    

In [13]:
ds2 = ds.drop(['Zp1','Z','Zl','Zu',
               'rA','rAw','rAs','rAz','dxG','dyG','Depth','PHrefF','PHrefC','drF','drC', 
              'hFacW','hFacS','hFacC','PhiBot','SIarea','SIheff','SIhsalt','SIhsnow','SIuice','SIvice',
              'Salt','Theta','W','oceFWflx','oceQnet','oceQsw','oceSflux'])

In [14]:
ds2

<xarray.Dataset>
Dimensions:  (XC: 480, XG: 480, YC: 680, YG: 680, time: 1680)
Coordinates:
  * YC       (YC) float32 -45.047726 -45.033997 -45.020264 -45.006527 ...
  * YG       (YG) float32 -45.054592 -45.040863 -45.02713 -45.013397 ...
  * XC       (XC) float32 5.03125 5.0520835 5.0729165 5.09375 5.1145835 ...
  * XG       (XG) float32 5.0208335 5.0416665 5.0625 5.0833335 5.1041665 ...
  * time     (time) float64 2.592e+05 2.628e+05 2.664e+05 2.7e+05 2.736e+05 ...
Data variables:
    face     (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
    dxC      (YC, XG) float32 ...
    dyC      (YG, XC) float32 ...
    Eta      (time, YC, XC) float32 dask.array<shape=(1680, 680, 480), chunksize=(1, 680, 480)>
    U        (time, YC, XG) float32 dask.array<shape=(1680, 680, 480), chunksize=(1, 680, 480)>
    V        (time, YG, XC) float32 dask.array<shape=(1680, 680, 480), chunksize=(1, 680, 480)>
    iter     (time) int64 dask.array<shape=(1680,), chunksize=(1,)>
    oceT

In [15]:
with ProgressBar():
    ds3 = ds2.load()

[########################################] | 100% Completed |  1hr 38min 31.7s


# llc velocity analysis#

## The Aim##
The aim is to train a deep neural net to infer velocities from the Sea surface heights ($\eta$) and Wind stress($\tau_x$ and $\tau_y$). The training dataset will be the high resolution model, We will then coarsegrain the model fields to get the lower resolution fields and use that as the testing data. The ultimate aim will be to get the velocity fields for Satellite altimetry. 

The hypothesis to be tested is the following:
Can we train a Conv Neural Net to give velocity estimates from altimetry data to get a better signture of small scale (balanced/unbalanced motions) than geostrophy?

## The present work##

In this notebook we calculate the surface velocity from the llc4320 model output in the Agulhas sector and write down the formalism for calculating the surface geostrophic velocities from the SSH ($\eta$) and the surface Ekman velocities from wind stress and the formalism for calculating the error.


The total momentum equation can be written as:

$$ \frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u} + f \times \mathbf{u} = -g \nabla \eta + \mathbf{F}$$

where $\mathbf{F}$ is the frictional term

Our traditional method involves splitting the surface flow into a geostrophic and an ageostrophic part as follows:

$$\mathbf{u} = \mathbf{u_g} + \mathbf{u_a}$$

where the force balances are 

$$ f \times \mathbf{u_g} = -g \nabla \eta$$
and
$$ f \times \mathbf{u_a} = F$$



# Geostrophic velocities#

Geostrophic velocities are given by 

$$fv_{g} = g \frac{\partial \eta}{\partial x} $$

$$fu_{g} = - g \frac{\partial \eta}{\partial y} $$



# Ekman velocity #


Under steady state conditions is can be shown that in the boundary layer of the upper ocean (order hundred meters) horizontal gradients are small compared to vertical gradients. Under these conditions, there is a balance between Coriolis and Friction.
Friction in the upper layer is provided by the wind stress.

$$ f v + \frac{\partial \tau_x}{\partial z} = 0$$

$$ f u - \frac{\partial \tau_y}{\partial z} = 0$$ (1)

and 

$$\tau_x = \rho A_z \frac{\partial u}{\partial z}$$

$$\tau_y = \rho A_z \frac{\partial v}{\partial z}$$ (2)


we write $[u,v]$ as a complex velocity as:
$$ \mathbf{u} = u + iv$$

We also write $[\tau_x, \tau_y]$ as a complex wind stress as:
$$ \mathbf{\tau} = \tau_x + i \tau_y$$

This gives us 
$$ \mathbf{u} = u + iv = (u,v)$$
$$ \mathbf{\underline{u}} = -v + iu = (-v,u)$$

$$ \mathbf{\tau} = \tau_x + i\tau_y = (\tau_x,\tau_y)$$
$$ \mathbf{\underline{\tau}} = -\tau_y + i\tau_x = (-\tau_y,\tau_x)$$

therefore the above equations can be written as 

$$  f \mathbf{u} = \frac{\partial }{\partial z} \mathbf{\underline{\tau}}$$

$$ \mathbf{\tau} = \rho A_z \frac{\partial \mathbf{u}}{\partial z} $$

So now using equations (2)

we re write the euqtions as

$$ f \mathbf{u}_{zz} = \frac{ i f }{A_z} \mathbf{u} $$

In the Northern Hemisphere $f > 0$. So the general solution is

$$ \mathbf{u} = \alpha_{+} e^{[(if/A_z)^{1/2}] z} + \alpha_{-} e^{[-(if/A_z)^{1/2}] z} $$

subject to the boundary conditions

$$ A_z {(u + i v)}_z = \tau_x^0 + i \tau_y^0 $$ at $z = 0$

and 

$$ (u + i v) = u_g + i v_g $$  at $z = -H$

For the Northern Hemisphere ($f>0$), the solution is written as

$$ \mathbf{u} = \alpha_{+} e^{[1+i] z/d} + \alpha_{-} e^{-[1+i] z/d} $$

Where the Ekman depth $d$ is

$$ d = \sqrt{\frac{2 A_z}{|f|}}$$

and $\alpha_{+}$ and $\alpha_{-}$ are complex coefficients. the $\alpha_{+}$ part denotes the solution decaying away from the top and the $\alpha_{-}$ denotes the solution decaying away from the bottom. For the surface Ekman velocities we are concerned with the $\alpha_{+}$ part.

For the Southern hemisphere the solution now becomes

$$ \mathbf{u} = \alpha_{+} e^{[-1+i] z/d} + \alpha_{-} e^{-[-1+i] z/d} $$

Here now the $\alpha_{-}$ part is the solution for the surface Ekman flow.

### Northern Hemisphere ###
To solve for $\alpha_{+}$, plug in $\mathbf{u}$ in the equation 

$$ \mathbf{\tau} = \rho A_z \frac{\partial \mathbf{u}}{\partial z} $$

This gives us at $z = 0$

$$ \alpha_{+} = \frac{(\tau_x +\tau_y) d}{\rho A_z (1+i)} = \frac{(\tau_x +\tau_y) (1-i) d}{2 \rho A_z }$$

$$ \implies \alpha_{+} = \frac{(\mathbf{\tau} - \mathbf{\underline{\tau}}) d}{2 \rho A_z}$$

So, we have

$$ u_{e} + i v_{e} = \frac{d}{2 \rho A_z} \left[ (\tau_x + i\tau_y) - (-\tau_y + i\tau_x)\right]$$

$$ \implies u_{e} = \frac{1}{\rho \sqrt{2 A_z |f|}} (\tau_x + \tau_y)$$
$$ \implies v_{e} = \frac{1}{\rho \sqrt{2 A_z |f|}} (-\tau_x + \tau_y)$$



### Southern Hemisphere ###

Doing the similar procedure in the Southern Hemisphere we get

$$ \implies \alpha_{-} = \frac{(\mathbf{\tau} + \mathbf{\underline{\tau}}) d}{2 \rho A_z}$$


So, we have

$$ u_{e} + i v_{e} = \frac{d}{2 \rho A_z} \left[ (\tau_x + i\tau_y) + (-\tau_y + i\tau_x)\right]$$

$$ \implies u_{e} = \frac{1}{\rho \sqrt{2 A_z |f|}} (\tau_x - \tau_y)$$
$$ \implies v_{e} = \frac{1}{\rho \sqrt{2 A_z |f|}} (\tau_x + \tau_y)$$
 



# Linear Regression #

For the first exercise, we aim to fit a multiple linear regression. For our black box therefore the input variables are $\left[x_{i1}, ... , x_{ip}\right]_{i=1}^{n}$, n being the number of samples, and p being the number of features. We can represent the linear regression problem as $u_i = \beta_0 1 + \beta_1 x_{i1} + ... + \beta_p x_{ip} + \epsilon_i$.

$$
X=
  \begin{bmatrix}
    1 & x_{11} & x_{12} ...  & x_{1p} \\
    1 & x_{21} & x_{22} ...  & x_{2p} \\
    .. & ..  & .... &...\\
    1 & x_{n1} & x_{n2} ...  & x_{np}
  \end{bmatrix}
$$

or 

$$
X = \begin{bmatrix}
    x_1^T \\
    x_2^T \\
    .. \\
    x_n^T
  \end{bmatrix}
$$

and 

$$
\beta = \begin{bmatrix}
    \beta_0 \\
    \beta_1 \\
    .. \\
    \beta_p
  \end{bmatrix}
$$

$$
U= \begin{bmatrix}
    u_0 \\
    u_1 \\
    .. \\
    u_n
  \end{bmatrix}
$$

$$
\epsilon = \begin{bmatrix}
    \epsilon_0 \\
    \epsilon_1 \\
    .. \\
    \epsilon_n
  \end{bmatrix}
$$

For our example our 9 features (variables) are $[f, \tau_x,\tau_y, \eta_{x+}, \eta_{x-}, \eta_{y+}, \eta_{y-}, \frac{1}{dx}, \frac{1}{dy}]$ .

where $U$ and $\beta$ are complex valued. For vectorization we consider them to have 2 columns each

$$
U = \begin{bmatrix}
    u_1 & v_1\\
    u_2 & v_2\\
    .. \\
    u_n & v_n
  \end{bmatrix}
$$

and 

$$
\beta = \begin{bmatrix}
        \beta_{0r} & \beta_{0i}\\
        \beta_{1r} & \beta_{1i}\\
        .. \\
        \beta_{pr} & \beta_{pi}
  \end{bmatrix}
$$

Our linear regression problem is therefore 
$$
\underbrace{\begin{bmatrix}
    u_1 & v_1\\
    u_2 & v_2\\
    .. \\
    u_n & v_n
  \end{bmatrix}}_{u = [n (samples) \times 2]} =
  \underbrace{\begin{bmatrix}
    1 & x_{11} & x_{12} ...  & x_{19} \\
    1 & x_{21} & x_{22} ...  & x_{29} \\
    .. & ..  & .... &...\\
    1 & x_{n1} & x_{n2} ...  & x_{n9}
  \end{bmatrix}}_{X = \left[n (samples) \times [9(features)+1]\right]} 
  \cdot
  \underbrace{\begin{bmatrix}
        \beta_{0r} & \beta_{0i}\\
        \beta_{1r} & \beta_{1i}\\
        .. \\
        \beta_{9r} & \beta_{9i}
  \end{bmatrix}}_{\beta = \left[[9(coefficients)+1(intercept)] \times 2\right]}
$$

here $[\beta_1, ..., \beta_9]$ are the coefficients and $\beta_0$ is the intercept.

In [28]:
utest = (grid.interp(ds3.U, 'X', boundary='extend')).isel(XC=slice(1,-1), YC=slice(1,-1))

In [29]:
utest

<xarray.DataArray (time: 1680, YC: 678, XC: 478)>
array([[[ 0.086156,  0.088829, ...,  0.061637,  0.059877],
        [ 0.082632,  0.085912, ...,  0.062075,  0.062684],
        ...,
        [ 0.010902,  0.000868, ...,  0.122118,  0.093484],
        [ 0.009129,  0.005507, ...,  0.108912,  0.104007]],

       [[ 0.073341,  0.075224, ..., -0.006776, -0.007682],
        [ 0.066512,  0.067344, ..., -0.009057, -0.007839],
        ...,
        [ 0.027807,  0.010653, ...,  0.137377,  0.098379],
        [ 0.025119,  0.014287, ...,  0.128164,  0.112167]],

       ...,

       [[ 0.235591,  0.233456, ...,  0.155558,  0.152526],
        [ 0.241239,  0.240167, ...,  0.139357,  0.134763],
        ...,
        [ 0.13092 ,  0.127579, ...,  0.370904,  0.376991],
        [ 0.116495,  0.114957, ...,  0.385301,  0.388683]],

       [[ 0.218607,  0.205631, ...,  0.14934 ,  0.151077],
        [ 0.217066,  0.206182, ...,  0.134799,  0.134606],
        ...,
        [ 0.164918,  0.161091, ...,  0.364046,  0.363

In [17]:
with ProgressBar():
    u = (grid.interp(ds3.U, 'X', boundary='extend')).isel(XC=slice(1,-1), YC=slice(1,-1)).values
    v = (grid.interp(ds3.V, 'Y', boundary='extend')).isel(XC=slice(1,-1), YC=slice(1,-1)).values

    tau_x = (grid.interp(ds3.oceTAUX, 'X', boundary='extend')).isel(XC=slice(1,-1), YC=slice(1,-1)).values
    tau_y = (grid.interp(ds3.oceTAUY, 'Y', boundary='extend')).isel(XC=slice(1,-1), YC=slice(1,-1)).values

    eta_xp = ds3.Eta.roll(XC=-1).isel(XC=slice(1,-1), YC=slice(1,-1)).values
    eta_xm = ds3.Eta.roll(XC=1).isel(XC=slice(1,-1), YC=slice(1,-1)).values
    eta_yp = ds3.Eta.roll(YC=-1).isel(XC=slice(1,-1), YC=slice(1,-1)).values
    eta_ym = ds3.Eta.roll(YC=1).isel(XC=slice(1,-1), YC=slice(1,-1)).values


In [27]:
u.shape, v.shape

((1680, 678, 478), (1680, 678, 478))

In [21]:
inv_dx_val = np.zeros_like(u, np.float32)
inv_dy_val = np.zeros_like(u, np.float32)

inv_dx_val[:] = ((1.0/(grid.interp(ds.dxC, 'X', boundary='extend'))).isel(XC=slice(1,-1), YC=slice(1,-1), drop=True)).values
inv_dy_val[:] = ((1.0/(grid.interp(ds.dyC, 'Y', boundary='extend'))).isel(XC=slice(1,-1), YC=slice(1,-1), drop=True)).values

Om = 7.2921e-5
f_c = np.abs(2 * Om * np.sin(np.deg2rad(ds.YC.isel(YC=slice(1,-1), drop=True)))).values

f_val = np.zeros_like(u, np.float32)
f_val[:] = f_c[np.newaxis,:,np.newaxis]

In [30]:
U = xr.DataArray(u,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

V = xr.DataArray(v,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Tau_x = xr.DataArray(tau_x,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Tau_y = xr.DataArray(tau_y,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Eta_xp = xr.DataArray(eta_xp,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Eta_xm = xr.DataArray(eta_xm,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Eta_yp = xr.DataArray(eta_yp,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

Eta_ym = xr.DataArray(eta_ym,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

f = xr.DataArray(f_val,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

inv_delx = xr.DataArray(inv_dx_val,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

inv_dely = xr.DataArray(inv_dy_val,
                 {'time': utest.time.values,
                  'YC': utest.YC.values,
                  'XC': utest.XC.values},
                 ('time', 'YC', 'XC'))

In [31]:
U

<xarray.DataArray (time: 1680, YC: 678, XC: 478)>
array([[[ 0.086156,  0.088829, ...,  0.061637,  0.059877],
        [ 0.082632,  0.085912, ...,  0.062075,  0.062684],
        ...,
        [ 0.010902,  0.000868, ...,  0.122118,  0.093484],
        [ 0.009129,  0.005507, ...,  0.108912,  0.104007]],

       [[ 0.073341,  0.075224, ..., -0.006776, -0.007682],
        [ 0.066512,  0.067344, ..., -0.009057, -0.007839],
        ...,
        [ 0.027807,  0.010653, ...,  0.137377,  0.098379],
        [ 0.025119,  0.014287, ...,  0.128164,  0.112167]],

       ...,

       [[ 0.235591,  0.233456, ...,  0.155558,  0.152526],
        [ 0.241239,  0.240167, ...,  0.139357,  0.134763],
        ...,
        [ 0.13092 ,  0.127579, ...,  0.370904,  0.376991],
        [ 0.116495,  0.114957, ...,  0.385301,  0.388683]],

       [[ 0.218607,  0.205631, ...,  0.14934 ,  0.151077],
        [ 0.217066,  0.206182, ...,  0.134799,  0.134606],
        ...,
        [ 0.164918,  0.161091, ...,  0.364046,  0.363

In [32]:
Eta_xp_ds = Eta_xp.to_dataset(name='eta_x_p')
Eta_xm_ds = Eta_xm.to_dataset(name='eta_x_m')
Eta_yp_ds = Eta_yp.to_dataset(name='eta_y_p')
Eta_ym_ds = Eta_ym.to_dataset(name='eta_y_m')

U_ds = U.to_dataset(name='U')
V_ds = V.to_dataset(name='V')

f_ds = f.to_dataset(name='f')
inv_dX_ds = inv_delx.to_dataset(name='inv_DX')
inv_dY_ds = inv_dely.to_dataset(name='inv_DY')

tau_X_ds = Tau_x.to_dataset(name='tau_x')
tau_Y_ds = Tau_x.to_dataset(name='tau_y')


In [33]:
test_data2 = xr.merge([f_ds, tau_X_ds, tau_Y_ds, Eta_xp_ds, Eta_xm_ds, Eta_yp_ds, Eta_ym_ds, inv_dX_ds, inv_dY_ds, U_ds, V_ds ])

In [34]:
outdir = '/swot/SUM02/analysis_ml_llc_agulhas/'
test_data2.to_netcdf(path=outdir +'Dataset_70days_10degbox.nc', engine='scipy')